# 1. Install and Import Dependencies

In [ ]:
!pip uninstall -y torch torchvision torchaudio


Found existing installation: torch 2.5.1+cu118
Uninstalling torch-2.5.1+cu118:
  Successfully uninstalled torch-2.5.1+cu118
Found existing installation: torchvision 0.20.1+cu124
Uninstalling torchvision-0.20.1+cu124:
  Successfully uninstalled torchvision-0.20.1+cu124
Found existing installation: torchaudio 2.5.1+cu124
Uninstalling torchaudio-2.5.1+cu124:
  Successfully uninstalled torchaudio-2.5.1+cu124


In [ ]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124

Looking in indexes: https://download.pytorch.org/whl/cu124


In [ ]:
!pip install transformers requests beautifulsoup4 pandas numpy

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re


# 2. Instantiate Model

In [ ]:
print(torch.__version__)


2.6.0+cu124


In [ ]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/669M [00:00<?, ?B/s]

# 3. Encode and Calculate Sentiment

In [ ]:
tokens = tokenizer.encode('Meh,it was okay!', return_tensors='pt')

In [ ]:
tokens

tensor([[  101, 10525, 10243,   117, 10197, 10140, 44810, 10158,   106,   102]])

In [ ]:
tokenizer.decode(tokens[0])

'[CLS] meh, it was okay! [SEP]'

In [ ]:
result = model(tokens)

In [ ]:
result.logits

tensor([[-1.1413,  1.5708,  3.0444, -0.0682, -2.8842]],
       grad_fn=<AddmmBackward0>)

In [ ]:
int(torch.argmax(result.logits))+1#position of the highest value
#between 1 to 5 with 1 the worst and 5 the best

3

# 4. Collect Reviews

In [ ]:
r = requests.get('https://www.trustpilot.com/review/bitcoin.com')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*typography_body-l__v5JLj typography_appearance-default__t8iAq typography_color-black__wpn7m.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [ ]:
reviews

['How does the profits not able to withdraw, there are so many scammers, claiming to help you withdraw your profit, but end up stealing money,when you pay using cash app to send the buying at their links address...Who controls the scams?',
 "As you know I'm new member and I'm participating in the business. But so far I didn't get even the bonus withdrowal. How can I proceed for withdrawal. Please help me. Your name is trust pilot. But your practice is not. I sent for you the screenshot of the platform. Your replay is nothing. As far as I used data at least I need some money. Your fake advertising says 5000$ for registration for criptomania.",
 "His justice, what about my justice. I didn't care for the fact that I purchase bitcoin and never received any if my rewards.  I believe that I was scammed and no one seems to care one way or another.  His justice, I WANT JUSTICE FOR ME!!!  What kind of more information you need? Do i send you copies of the information when I purchased my biticoi

# 5. Load Reviews into DataFrame and Score

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [ ]:
df['review'].iloc[0]

'How does the profits not able to withdraw, there are so many scammers, claiming to help you withdraw your profit, but end up stealing money,when you pay using cash app to send the buying at their links address...Who controls the scams?'

In [ ]:
df

,review
0,"How does the profits not able to withdraw, the..."
1,As you know I'm new member and I'm participati...
2,"His justice, what about my justice. I didn't c..."
3,Great reading for someone who feels like a rab...
4,I really want to know more about cryptocurrenc...
5,I write this as a very *URGENT* request to log...
6,thank you for google inc and LLC or Company Gr...
7,Everything about this company special ✨️ very ...
8,I was having a problem recovering my wallet an...
9,Nothing ok I want to see my account verified


In [ ]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [ ]:
sentiment_score(df['review'].iloc[1])

1

In [ ]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))#*ax tokens=512

In [ ]:
df

,review,sentiment
0,"How does the profits not able to withdraw, the...",1
1,As you know I'm new member and I'm participati...,1
2,"His justice, what about my justice. I didn't c...",1
3,Great reading for someone who feels like a rab...,5
4,I really want to know more about cryptocurrenc...,5
5,I write this as a very *URGENT* request to log...,1
6,thank you for google inc and LLC or Company Gr...,5
7,Everything about this company special ✨️ very ...,5
8,I was having a problem recovering my wallet an...,5
9,Nothing ok I want to see my account verified,2


In [ ]:
df['review'].iloc[3]

'Great reading for someone who feels like a rabbit in front of car headlights when it comes to crypto world. '

#REDIT


In [ ]:
!pip install praw


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.3/189.3 kB 4.1 MB/s eta 0:00:00


In [ ]:
import praw
import pandas as pd
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

# Download NLTK VADER model
nltk.download('vader_lexicon')

# Initialize Sentiment Analyzer
sia = SentimentIntensityAnalyzer()

# Reddit API Credentials (replace with your credentials)
reddit = praw.Reddit(
    client_id='boIus-SkhJUeorHNhyPaaQ',
    client_secret='VrwReHJBK6SL_BgGiMktd2AWgGO2XQ',
    user_agent='SentimentAn'
)

# Function to fetch Reddit comments about a cryptocurrency
def fetch_reddit_comments(crypto_name, limit=100):
    subreddit = reddit.subreddit("cryptocurrency")
    comments = []
    for submission in subreddit.search(crypto_name, sort='new', limit=10):
        submission.comments.replace_more(limit=0)
        for comment in submission.comments.list()[:limit]:
            comments.append(comment.body)
    return comments

# Function to analyze sentiment
def analyze_sentiment(text):
    return sia.polarity_scores(text)['compound']

# Function to get average sentiment score for a cryptocurrency
def get_crypto_sentiment(crypto_name, limit=100):
    comments = fetch_reddit_comments(crypto_name, limit)
    sentiments = [analyze_sentiment(comment) for comment in comments]
    return sum(sentiments) / len(sentiments) if sentiments else 0

# Example usage
cryptos = ["Bitcoin", "Ethereum", "Solana"]
sentiment_scores = {crypto: get_crypto_sentiment(crypto) for crypto in cryptos}

# Convert to DataFrame for better visualization
sentiment_df = pd.DataFrame(sentiment_scores.items(), columns=['Cryptocurrency', 'Sentiment Score'])
print(sentiment_df)


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: ht

  Cryptocurrency  Sentiment Score
0        Bitcoin         0.129157
1       Ethereum         0.050772
2         Solana         0.151646


In [ ]:
import praw
import pandas as pd
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

# Download NLTK VADER model
nltk.download('vader_lexicon')

# Initialize Sentiment Analyzer
sia = SentimentIntensityAnalyzer()

# Reddit API Credentials (replace with your credentials)
reddit = praw.Reddit(
    client_id='boIus-SkhJUeorHNhyPaaQ',
    client_secret='VrwReHJBK6SL_BgGiMktd2AWgGO2XQ',
    user_agent='SentimentAn'
)

# Function to fetch Reddit comments about a cryptocurrency
def fetch_reddit_comments(crypto_name, limit=100):
    subreddit = reddit.subreddit("cryptocurrency")
    comments = []
    for submission in subreddit.search(crypto_name, sort='new', limit=10):
        submission.comments.replace_more(limit=0)
        for comment in submission.comments.list()[:limit]:
            comments.append(comment.body)
    return comments

# Existing sentiment analysis function (adapted for Reddit comments)
def analyze_sentiment_old_function(text):
    # Reuse your old function here, modifying it to process Reddit comments
    return sia.polarity_scores(text)['compound']

# Function to get average sentiment score for a cryptocurrency
def get_crypto_sentiment(crypto_name, limit=100):
    comments = fetch_reddit_comments(crypto_name, limit)
    sentiments = [analyze_sentiment_old_function(comment) for comment in comments]
    return sum(sentiments) / len(sentiments) if sentiments else 0

# Example usage
cryptos = ["Bitcoin", "Ethereum", "Solana"]
sentiment_scores = {crypto: get_crypto_sentiment(crypto) for crypto in cryptos}

# Convert to DataFrame for better visualization
sentiment_df = pd.DataFrame(sentiment_scores.items(), columns=['Cryptocurrency', 'Sentiment Score'])
print(sentiment_df)


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for m

  Cryptocurrency  Sentiment Score
0        Bitcoin         0.132486
1       Ethereum         0.000000
2         Solana         0.000000


In [ ]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')  # Tokenizing the review
    result = model(tokens)  # Getting the model's output
    sentiment_class = int(torch.argmax(result.logits))  # Getting the sentiment class index (0 to 4)
    return sentiment_class + 1  # Shifting to a 1-5 scale


In [ ]:
import praw
import pandas as pd
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

# Download NLTK VADER model
nltk.download('vader_lexicon')

# Initialize Sentiment Analyzer
sia = SentimentIntensityAnalyzer()

# Reddit API Credentials (replace with your credentials)
reddit = praw.Reddit(
    client_id='boIus-SkhJUeorHNhyPaaQ',
    client_secret='VrwReHJBK6SL_BgGiMktd2AWgGO2XQ',
    user_agent='SentimentAn'
)

def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')  # Tokenizing the review
    result = model(tokens)  # Getting the model's output
    sentiment_class = int(torch.argmax(result.logits))  # Getting the sentiment class index (0 to 4)
    return sentiment_class + 1  # Shifting to a 1-5 scale

# Function to fetch Reddit comments about a cryptocurrency
def fetch_reddit_comments(crypto_name, limit=100):
    subreddit = reddit.subreddit("cryptocurrency")
    comments = []
    for submission in subreddit.search(crypto_name, sort='new', limit=10):
        submission.comments.replace_more(limit=0)
        for comment in submission.comments.list()[:limit]:
            comments.append(comment.body)
    return comments

# Existing sentiment analysis function (adapted for Reddit comments)
def analyze_sentiment_old_function(text):
    # Reuse your old function here, modifying it to process Reddit comments
    return sia.polarity_scores(text)['compound']

# Function to get average sentiment score for a cryptocurrency
def get_crypto_sentiment(crypto_name, limit=100):
    comments = fetch_reddit_comments(crypto_name, limit)
    sentiments = [analyze_sentiment_old_function(comment) for comment in comments]

    # Print a few comments for demonstration
    print(f"Some comments for {crypto_name}:")
    for comment in comments[:5]:  # Print first 5 comments
        print(f"- {comment}")
    print("\n")

    return sum(sentiments) / len(sentiments) if sentiments else 0

# Example usage
cryptos = ["Bitcoin", "Ethereum", "Solana"]
sentiment_scores = {crypto: get_crypto_sentiment(crypto) for crypto in cryptos}

# Convert to DataFrame for better visualization
sentiment_df = pd.DataFrame(sentiment_scores.items(), columns=['Cryptocurrency', 'Sentiment Score'])
print(sentiment_df)


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for m

Some comments for Bitcoin:
- I've dealt with a couple corrupted hard drives. It takes so little for them to get corrupted. Mine were in neoprene cases and no idea what caused the corruption but at least mine were just movies.

After being unprotected in a landfill for this long, it's a pipedream to even think the stick still has data on it, let alone finding it in the first place.
- Can this guy just go away.  After the last failed court case I kinda hoped that would be the end. We are still going to be hearing about hard drive bitcoin guy in 2050 aren't we?
- This could be a little fable, man with a decent life decides to live in a landfill in hopes to find a better life but he ends up searching the landfill until he dies
- This guy is going to get a movie made about him and I can’t yet tell if it’s going to be as dark as There Will Be Blood or as tone-deaf inspirational like The Blind Side. 
- That hard drive has probably been found in 2015 by anyone else and those coins are far gone

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/l

Some comments for Ethereum:
- Be advised, the website dailycoin.com has proven to be an unreliable source of information. Please verify/fact-check the information in the article from independent sources before relying on it or coming to any conclusions. 

*I am a bot, and this action was performed automatically. Please [contact the moderators of this subreddit](/message/compose/?to=/r/CryptoCurrency) if you have any questions or concerns.*
- Fucking hell. Its gonna be a long hold for me
- Binance sold all their ETH, SOL and 94% of their BTC. Then said your funds are SAFU.
- tldr; Ethereum experienced a sudden crash, losing over 36% of its value, attributed to a manipulated market event. The crash was triggered by a combination of overleveraged Ethereum long futures positions and aggressive put option buying, leading to significant delta hedging. This was exacerbated by US President Trump's tariff hike announcement, which market makers used as a cover to execute a controlled liquidation

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/l

Some comments for Solana:
- tldr; The USDC stablecoin has reached a new all-time high (ATH) in market cap, driven by increased adoption and market dynamics. USDC, issued by Circle, is a US-based and regulated stablecoin that has been gaining traction as a competitor to Tether (USDT). Despite a setback in March 2023 when it temporarily lost its peg due to a bank failure, USDC has seen growth in 2024, with a significant increase in supply and market cap, reaching 56 billion. The stablecoin's allocation is primarily on Ethereum, with Solana gaining a notable share. USDC's market share against Tether has been recovering, reaching nearly 30%.

*This summary is auto generated by a bot and not meant to replace reading the original article. As always, DYOR.
- Why use stable coins instead of just letting your fiat on your wallet ? Your fiat is supposed to be quite stable no ?  
Or is it only to trade on dex ?
- It looks like this post is about taxes.  Tax laws vary between countries, so you may

In [ ]:
import praw
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Initialize tokenizer and model (assuming a 5-class sentiment model)
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
model = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=5)

# Function to get sentiment score on a scale of 1 to 5
def sentiment_score(review):
    # Tokenize the input review text with truncation to ensure the input is no longer than 512 tokens
    tokens = tokenizer.encode(review, return_tensors='pt', truncation=True, max_length=512)

    # Get the model's output logits
    result = model(tokens)

    # Get the index of the highest logit (the predicted sentiment class)
    sentiment_class = int(torch.argmax(result.logits))  # Index of highest logit (0 to 4)

    # Map this index to a 1-5 scale (adding 1 to shift it)
    return sentiment_class + 1  # Result will be from 1 to 5

# Reddit API Credentials (replace with your credentials)
reddit = praw.Reddit(
    client_id='boIus-SkhJUeorHNhyPaaQ',
    client_secret='VrwReHJBK6SL_BgGiMktd2AWgGO2XQ',
    user_agent='SentimentAn'
)

# Function to fetch Reddit comments about a cryptocurrency
def fetch_reddit_comments(crypto_name, limit=100):
    subreddit = reddit.subreddit("cryptocurrency")
    comments = []
    for submission in subreddit.search(crypto_name, sort='new', limit=10):
        submission.comments.replace_more(limit=0)
        for comment in submission.comments.list()[:limit]:
            comments.append(comment.body)
    return comments

# Function to get average sentiment score for a cryptocurrency
def get_crypto_sentiment(crypto_name, limit=100):
    comments = fetch_reddit_comments(crypto_name, limit)
    sentiments = [sentiment_score(comment) for comment in comments]

    # Print a few comments for demonstration
    print(f"Some comments for {crypto_name}:")
    for comment in comments[:5]:  # Print first 5 comments
        print(f"- {comment}")
    print("\n")

    return sum(sentiments) / len(sentiments) if sentiments else 0

# Example usage
cryptos = ["Bitcoin", "Ethereum", "Solana"]
sentiment_scores = {crypto: get_crypto_sentiment(crypto) for crypto in cryptos}

# Convert to DataFrame for better visualization
sentiment_df = pd.DataFrame(sentiment_scores.items(), columns=['Cryptocurrency', 'Sentiment Score'])
print(sentiment_df)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments 

Some comments for Bitcoin:
- I've dealt with a couple corrupted hard drives. It takes so little for them to get corrupted. Mine were in neoprene cases and no idea what caused the corruption but at least mine were just movies.

After being unprotected in a landfill for this long, it's a pipedream to even think the stick still has data on it, let alone finding it in the first place.
- This could be a little fable, man with a decent life decides to live in a landfill in hopes to find a better life but he ends up searching the landfill until he dies
- Can this guy just go away.  After the last failed court case I kinda hoped that would be the end. We are still going to be hearing about hard drive bitcoin guy in 2050 aren't we?
- This guy is going to get a movie made about him and I can’t yet tell if it’s going to be as dark as There Will Be Blood or as tone-deaf inspirational like The Blind Side. 
- That hard drive has probably been found in 2015 by anyone else and those coins are far gone

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/l

Some comments for Ethereum:
- Be advised, the website dailycoin.com has proven to be an unreliable source of information. Please verify/fact-check the information in the article from independent sources before relying on it or coming to any conclusions. 

*I am a bot, and this action was performed automatically. Please [contact the moderators of this subreddit](/message/compose/?to=/r/CryptoCurrency) if you have any questions or concerns.*
- Fucking hell. Its gonna be a long hold for me
- Binance sold all their ETH, SOL and 94% of their BTC. Then said your funds are SAFU.
- tldr; Ethereum experienced a sudden crash, losing over 36% of its value, attributed to a manipulated market event. The crash was triggered by a combination of overleveraged Ethereum long futures positions and aggressive put option buying, leading to significant delta hedging. This was exacerbated by US President Trump's tariff hike announcement, which market makers used as a cover to execute a controlled liquidation

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/l

Some comments for Solana:
- tldr; The USDC stablecoin has reached a new all-time high (ATH) in market cap, driven by increased adoption and market dynamics. USDC, issued by Circle, is a US-based and regulated stablecoin that has been gaining traction as a competitor to Tether (USDT). Despite a setback in March 2023 when it temporarily lost its peg due to a bank failure, USDC has seen growth in 2024, with a significant increase in supply and market cap, reaching 56 billion. The stablecoin's allocation is primarily on Ethereum, with Solana gaining a notable share. USDC's market share against Tether has been recovering, reaching nearly 30%.

*This summary is auto generated by a bot and not meant to replace reading the original article. As always, DYOR.
- Why use stable coins instead of just letting your fiat on your wallet ? Your fiat is supposed to be quite stable no ?  
Or is it only to trade on dex ?
- It looks like this post is about taxes.  Tax laws vary between countries, so you may